In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request 
import csv
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
try:
    df = pd.read_csv('character_names.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('character_names.csv', encoding='ISO-8859-1')
    except UnicodeDecodeError:
        df = pd.read_csv('character_names.csv', encoding='cp1252')

df.head()

,Id,Character,Occupation
0,1,Aaron Taylor-Johnson,model
1,2,Adam Levine,model
2,3,Adrian Brody,model
3,4,Aiden Turner,model
4,5,Alexander Skarsgård,model


In [4]:
df['base_query'] = df['Occupation'] + ' ' + df['Character']
df.head()

,Id,Character,Occupation,base_query
0,1,Aaron Taylor-Johnson,model,model Aaron Taylor-Johnson
1,2,Adam Levine,model,model Adam Levine
2,3,Adrian Brody,model,model Adrian Brody
3,4,Aiden Turner,model,model Aiden Turner
4,5,Alexander Skarsgård,model,model Alexander Skarsgård


In [ ]:
extras = ['real', 'wallpaper', ]
for _ in extras:
    df['base_query_' + _ ] = df['base_query'] + ' ' + _

df.head()

,Id,Character,Occupation,base_query,base_query_real,base_query_wallpaper
0,1,Aaron Taylor-Johnson,model,model Aaron Taylor-Johnson,model Aaron Taylor-Johnson real,model Aaron Taylor-Johnson wallpaper
1,2,Adam Levine,model,model Adam Levine,model Adam Levine real,model Adam Levine wallpaper
2,3,Adrian Brody,model,model Adrian Brody,model Adrian Brody real,model Adrian Brody wallpaper
3,4,Aiden Turner,model,model Aiden Turner,model Aiden Turner real,model Aiden Turner wallpaper
4,5,Alexander Skarsgård,model,model Alexander Skarsgård,model Alexander Skarsgård real,model Alexander Skarsgård wallpaper


In [6]:
def create_query_list(row):
    return [row['base_query'], row['base_query_real'], row['base_query_wallpaper']]

df['queries'] = df.apply(create_query_list, axis=1)

df.head()

,Id,Character,Occupation,base_query,base_query_real,base_query_wallpaper,queries
0,1,Aaron Taylor-Johnson,model,model Aaron Taylor-Johnson,model Aaron Taylor-Johnson real,model Aaron Taylor-Johnson wallpaper,"[model Aaron Taylor-Johnson, model Aaron Taylo..."
1,2,Adam Levine,model,model Adam Levine,model Adam Levine real,model Adam Levine wallpaper,"[model Adam Levine, model Adam Levine real, mo..."
2,3,Adrian Brody,model,model Adrian Brody,model Adrian Brody real,model Adrian Brody wallpaper,"[model Adrian Brody, model Adrian Brody real, ..."
3,4,Aiden Turner,model,model Aiden Turner,model Aiden Turner real,model Aiden Turner wallpaper,"[model Aiden Turner, model Aiden Turner real, ..."
4,5,Alexander Skarsgård,model,model Alexander Skarsgård,model Alexander Skarsgård real,model Alexander Skarsgård wallpaper,"[model Alexander Skarsgård, model Alexander Sk..."


In [12]:
df.head()

,Id,Character,Occupation,base_query,base_query_real,base_query_wallpaper,queries
0,1,Aaron Taylor-Johnson,model,model Aaron Taylor-Johnson,model Aaron Taylor-Johnson real,model Aaron Taylor-Johnson wallpaper,"[model Aaron Taylor-Johnson, model Aaron Taylo..."
1,2,Adam Levine,model,model Adam Levine,model Adam Levine real,model Adam Levine wallpaper,"[model Adam Levine, model Adam Levine real, mo..."
2,3,Adrian Brody,model,model Adrian Brody,model Adrian Brody real,model Adrian Brody wallpaper,"[model Adrian Brody, model Adrian Brody real, ..."
3,4,Aiden Turner,model,model Aiden Turner,model Aiden Turner real,model Aiden Turner wallpaper,"[model Aiden Turner, model Aiden Turner real, ..."
4,5,Alexander Skarsgård,model,model Alexander Skarsgård,model Alexander Skarsgård real,model Alexander Skarsgård wallpaper,"[model Alexander Skarsgård, model Alexander Sk..."


In [14]:
# Remove leading/trailing whitespaces and tabs from column names
df.columns = df.columns.str.strip()

# Drop the columns after cleaning the names
df = df.drop(['base_query', 'base_query_real', 'base_query_wallpaper'], axis=1)
df.head()


,Id,Character,Occupation,queries
0,1,Aaron Taylor-Johnson,model,"[model Aaron Taylor-Johnson, model Aaron Taylo..."
1,2,Adam Levine,model,"[model Adam Levine, model Adam Levine real, mo..."
2,3,Adrian Brody,model,"[model Adrian Brody, model Adrian Brody real, ..."
3,4,Aiden Turner,model,"[model Aiden Turner, model Aiden Turner real, ..."
4,5,Alexander Skarsgård,model,"[model Alexander Skarsgård, model Alexander Sk..."


In [10]:
print(df.columns)


Index(['Id', 'Character', 'Occupation', 'base_query', 'base_query_real',
       'base_query_wallpaper', 'queries'],
      dtype='object')


In [15]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

# Function to get image URLs for a given search query
def get_image_urls(search_query):
    search_url = 'https://www.bing.com/images/search'
    params = {'q': search_query}
    response = requests.get(search_url, params=params)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        image_tags = soup.find_all('img', class_=lambda x: x and 'mimg' in x)
        # Extract 'src' or 'data-src' attribute based on which is available
        image_srcs = [img['src'] if img.has_attr('src') else img['data-src'] for img in image_tags if img.has_attr('src') or img.has_attr('data-src')]
        return image_srcs[:50]  # Return up to 50 image source URLs
    else:
        print(f'An error occurred while searching for: {search_query}')
        return []  # Return an empty list in case of an error

# Assuming df is your DataFrame and it already contains a 'queries' column with lists of queries
# We'll first define a new column 'urls' with empty lists
df['urls'] = pd.Series([[] for _ in range(len(df))], index=df.index)

# Iterate over the DataFrame rows with a progress bar
for index in tqdm(df.index, desc='Processing Characters'):
    all_image_urls = []
    row = df.loc[index]
    # For each query in the list of queries, perform a search and collect image URLs
    for query in row['queries']:
        image_urls = get_image_urls(query)
        all_image_urls.extend(image_urls)
        # Stop if we've collected 50 URLs, no need for further searches
        if len(all_image_urls) >= 50:
            break

    # Assign up to 50 URLs to the 'urls' column for this row
    df.at[index, 'urls'] = all_image_urls[:50]

# Display the DataFrame to verify the results
print(df)


Processing Characters:   0%|          | 0/200 [00:00<?, ?it/s]

      Id             Character Occupation  \
0      1  Aaron Taylor-Johnson      model   
1      2           Adam Levine      model   
2      3          Adrian Brody      model   
3      4          Aiden Turner      model   
4      5   Alexander Skarsgård      model   
..   ...                   ...        ...   
195  196       Maria Sharapova      model   
196  197           Marie Curie      model   
197  198        Marilyn Monroe      model   
198  199        Martha Stewart      model   
199  200       Mary-Kate Olsen      model   

                                               queries  \
0    [model Aaron Taylor-Johnson, model Aaron Taylo...   
1    [model Adam Levine, model Adam Levine real, mo...   
2    [model Adrian Brody, model Adrian Brody real, ...   
3    [model Aiden Turner, model Aiden Turner real, ...   
4    [model Alexander Skarsgård, model Alexander Sk...   
..                                                 ...   
195  [model Maria Sharapova, model Maria Sharapova ..

In [16]:
len(df['urls'][0])

50

In [17]:
df['urls']

0      [https://tse4.mm.bing.net/th/id/OIP.y4xycaXVVE...
1      [https://tse1.mm.bing.net/th/id/OIP.fGhMJsCBeH...
2      [https://tse4.mm.bing.net/th/id/OIP.XXR_GJvU-Q...
3      [https://tse4.mm.bing.net/th/id/OIP.KJ4pGZVypJ...
4      [https://tse1.mm.bing.net/th/id/OIP.sZHXzdrQPC...
                             ...                        
195    [https://tse2.mm.bing.net/th/id/OIP.Ya1H2mDIVT...
196    [https://tse4.mm.bing.net/th/id/OIP.E8qz7UNfsm...
197    [https://tse2.mm.bing.net/th/id/OIP.EEj6QxM4Rw...
198    [https://tse4.mm.bing.net/th/id/OIP.LejKuJkJ0R...
199    [https://tse4.mm.bing.net/th/id/OIP.lL7Twd59KL...
Name: urls, Length: 200, dtype: object

In [18]:
import os
import requests
from tqdm.auto import tqdm

# Base directory where character folders will be created
base_dir = 'character_images'

# Ensure the base directory exists
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Function to download an image from a URL
def download_image(url, folder_path, image_number):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            # Create a valid filename from the image number
            filename = f'image_{image_number:03}.jpg'
            filepath = os.path.join(folder_path, filename)
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            return True
    except Exception as e:
        print(f"An error occurred: {e}")
    return False

# Iterate over the DataFrame rows and download images
for index, row in df.iterrows():
    character = row['Character']
    # Make a valid directory name for the character
    character_dir = os.path.join(base_dir, character.replace(' ', '_'))
    # Ensure the character's directory exists
    if not os.path.exists(character_dir):
        os.makedirs(character_dir)
    
    # Download each image from the 'urls' list
    for image_number, url in enumerate(tqdm(row['urls'], desc=f"Downloading images for {character}")):
        download_image(url, character_dir, image_number)


An error occurred: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [24]:
resume_row_id = 216

for index, row in df.iterrows():
    if index >= resume_row_id:
        character = row['Character']
        # Replace characters that are invalid for directory names
        safe_character = ''.join(c for c in character if c.isalnum() or c in (' ','_')).rstrip()
        character_dir = os.path.join(base_dir, safe_character.replace(' ', '_'))
        
        # Ensure the character's directory exists
        if not os.path.exists(character_dir):
            os.makedirs(character_dir)
        
        # Get the list of images that have already been downloaded to avoid re-downloading
        existing_images = set(os.listdir(character_dir))
        
        # Download each image from the 'urls' list
        for image_number, url in enumerate(tqdm(row['urls'], desc=f"Downloading images for {character}")):
            # Check if the image already exists
            image_filename = f'image_{image_number:03}.jpg'
            if image_filename not in existing_images:
                if download_image(url, character_dir, image_number):
                    existing_images.add(image_filename) 